In [3]:
import os
import xlwings as xw
import pandas as pd
import re

main_path = r'D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE'
os.chdir(main_path)

#获取工作列表脚本
def get_file_list(main_path):
    files_path = []
    for root, dirs, files in os.walk(main_path):
        for file in files:
            if file.endswith(('.csv', '.xls', '.xlsx')) and not file.startswith('~$'):
                files_path.append(os.path.join(root, file))
    return files_path

def process_file(file):
    #AKSHARE的期货列名对应关系字典
    futures_column_mapping = {
    "合约" : "symbol",
    "日期" : "date",
    "开盘价" : "open",
    "最高价" : "high",
    "最低价" : "low",
    "收盘价" : "close",
    "成交量" : "volume",
    "持仓量" : "open_interest",
    "成交金额" : "turnover",
    "结算价" : "settle",
    "前结算" : "pre_settle",
    }

    try:
        df = pd.read_excel(file, header=2)
        df['合约'] = df['合约'].ffill()
        df = df.drop(df[df['合约'].str.contains('注', na = False)].index)
        df = df.dropna(subset=['日期'])
        futures_columns_to_keep = list(futures_column_mapping.keys())
        df = df[futures_columns_to_keep]
        #更改列名
        df = df.rename(columns = futures_column_mapping)
        #创建Variety列
        df['variety'] = df['symbol'].str.extract(r'([a-zA-Z]+)')
        print(f'文件{file}处理完成!')
        return df
    except Exception as e:
        print(f'文件{file}出错{e}')
        return None

def table_combine(tables):
    tables = pd.concat(tables, ignore_index=True)
    date_error_rows = []
    for index, row in tables.iterrows():
        if not re.match(r'^\d{8}$', str(row['date'])):  # 使用正则表达式匹配8位数字日期
            date_error_rows.append(index)
    for i in date_error_rows:
        tables = tables.drop([i])
    futures = tables[tables['symbol'].str.match(r'^([a-z]+)\d+$')]  # 匹配'al0501'样式的合约代码
    options = tables[tables['symbol'].str.match(r'^([a-z]+)\d+[CP]\d+$')] #匹配'al0501C45000,al0501P45000'样式合约代码
    options.to_csv('SHFE_options.csv', index=False, encoding='gbk')
    print('期权数据 SHFE_options.csv 已保存!')
    futures.to_csv('SHFE_futures.csv', index=False, encoding='gbk')
    print('期货数据 SHFE_futures.csv 已保存!')

files = get_file_list(main_path)
tables = [process_file(file) for file in files]
table_combine(tables)

文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2009.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2010.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2011.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2012.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2013.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2014.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2015.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2016.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2017.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2018.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2019.1.xls处理完成!
文件D:\JupyterLabFiles\Futures\Futures_exchange_data\SHFE\Datas\所内合约行情报表2019

In [4]:
tables = pd.concat(tables, ignore_index=True)

In [9]:
import re
date_error_rows = []
for index, row in tables.iterrows():
    if not re.match(r'^\d{8}$', str(row['date'])):  # 使用正则表达式匹配8位数字日期
        date_error_rows.append(index)

In [16]:
for i in date_error_rows:
    tables = tables.drop([i])

In [21]:
futures = tables[tables['symbol'].str.match(r'^([a-z]+)\d+$')]  # 匹配'al0501'样式的合约代码
options = tables[tables['symbol'].str.match(r'^([a-z]+)\d+[CP]\d+$')]

In [18]:
futures.to_csv(

,symbol,date,open,high,low,close,volume,open_interest,turnover,settle,pre_settle,variety
0,al0901,20090105,12120.0,12140.0,11700.0,12030.0,3350.0,14040.0,20045.525,11965.0,11520.0,al
1,al0901,20090106,11800.0,12500.0,11800.0,12280.0,4310.0,13610.0,26216.025,12165.0,11965.0,al
2,al0901,20090107,12530.0,12690.0,12290.0,12330.0,1540.0,13570.0,9665.925,12550.0,12165.0,al
3,al0901,20090108,12205.0,12220.0,11920.0,11920.0,1960.0,13420.0,11807.375,12045.0,12550.0,al
4,al0901,20090109,12000.0,12280.0,12000.0,12280.0,1710.0,13140.0,10416.75,12180.0,12045.0,al
...,...,...,...,...,...,...,...,...,...,...,...,...
3038453,zn2509,20240924,23125,23315,23125,23315,24,31,278.1175,23175,23040,zn
3038454,zn2509,20240925,23595,24150,23595,23800,14,34,167.0825,23865,23175,zn
3038455,zn2509,20240926,23825,23880,23715,23715,13,35,154.615,23785,23865,zn
3038456,zn2509,20240927,24190,24420,24190,24230,34,43,412.9875,24290,23785,zn


In [22]:
options

,symbol,date,open,high,low,close,volume,open_interest,turnover,settle,pre_settle,variety
311208,cu1901C45000,20181220,3232.0,3426.0,2838.0,3257.0,732.0,68.0,1148.297,3159.0,3359.0,cu
311209,cu1901C45000,20181221,3359.0,3528.0,3210.0,3371.0,248.0,50.0,414.366,3319.0,3159.0,cu
311210,cu1901C45000,20181224,3457.0,3457.0,3095.0,3129.0,318.0,64.0,506.304,3210.0,3319.0,cu
311211,cu1901C46000,20180921,3760.0,4024.0,3700.0,3792.0,1568.0,182.0,3033.983,3792.0,3636.0,cu
311212,cu1901C46000,20180925,4207.0,4278.0,4020.0,4276.0,782.0,456.0,1607.574,4147.0,3792.0,cu
...,...,...,...,...,...,...,...,...,...,...,...,...
3042359,zn2501P27000,20240926,NaN,NaN,NaN,2990,0,0,0,2990,2961,zn
3042360,zn2501P27000,20240927,NaN,NaN,NaN,2601,0,0,0,2601,2990,zn
3042361,zn2501P27000,20240930,2489,2489,2489,2489,1,1,1.2445,2705,2601,zn
3042362,zn2501P27500,20240930,2904,2904,2904,2904,1,1,1.452,3092,3008,zn


In [20]:
tables

,symbol,date,open,high,low,close,volume,open_interest,turnover,settle,pre_settle,variety
0,al0901,20090105,12120.0,12140.0,11700.0,12030.0,3350.0,14040.0,20045.525,11965.0,11520.0,al
1,al0901,20090106,11800.0,12500.0,11800.0,12280.0,4310.0,13610.0,26216.025,12165.0,11965.0,al
2,al0901,20090107,12530.0,12690.0,12290.0,12330.0,1540.0,13570.0,9665.925,12550.0,12165.0,al
3,al0901,20090108,12205.0,12220.0,11920.0,11920.0,1960.0,13420.0,11807.375,12045.0,12550.0,al
4,al0901,20090109,12000.0,12280.0,12000.0,12280.0,1710.0,13140.0,10416.75,12180.0,12045.0,al
...,...,...,...,...,...,...,...,...,...,...,...,...
3042359,zn2501P27000,20240926,NaN,NaN,NaN,2990,0,0,0,2990,2961,zn
3042360,zn2501P27000,20240927,NaN,NaN,NaN,2601,0,0,0,2601,2990,zn
3042361,zn2501P27000,20240930,2489,2489,2489,2489,1,1,1.2445,2705,2601,zn
3042362,zn2501P27500,20240930,2904,2904,2904,2904,1,1,1.452,3092,3008,zn


In [23]:
options.to_csv('SHFE_options.csv', index=False, encoding='gbk')
print('期权数据 SHFE_options.csv 已保存!')
futures.to_csv('SHFE_futures.csv', index=False, encoding='gbk')
print('期货数据 SHFE_futures.csv 已保存')

期权数据 SHFE_options.csv 已保存!
期货数据 SHFE_futures.csv 已保存
